In [51]:
import os
import logging
logging.basicConfig(filename='pdf_parse_log.log',level=logging.DEBUG)

In [16]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO

In [19]:
path = "./scraped_docs/"
doc_paths = [path + x for x in os.listdir(path) if ".pdf" in x.lower()]

In [45]:
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = file(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        #print page
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [52]:
save_loc = "./text_parsed_docs/"
for doc in doc_paths:
    save_name = save_loc + doc.lower().replace(path, "").replace("pdf", "txt")
    with open(save_name, "w") as f:
        raw_text = convert_pdf_to_txt(doc)
        if len(raw_text) < 500:
            logging.warning("Doc: {} is too short ({} characters), check doc for integrity".format(save_name, len(raw_text)))
        f.write(raw_text)